In [115]:
#!pip install pydealer


     |████████████████████████████████| 170 kB 24.7 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=d66d6af94f31f361dfa0ea9189958920eb0d74044897ef906f5d8dfae607e048
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [139]:
import pydealer

class Card(object):

    card_values = {
        'Ace': 11,
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
        '10': 10,
        'Jack': 10,
        'Queen': 10,
        'King': 10
    }

    def __init__(self, suit, value):
        self.suit = suit.capitalize()
        self.value = value
        self.points = self.card_values[value]

def ascii_version_of_card(cards, add_inv = False):
    suits_name = ['Spades', 'Diamonds', 'Hearts', 'Clubs']
    suits_symbols = ['♠', '♦', '♥', '♣']
    lines = [[] for i in range(9)]

    for index, card in enumerate(cards):
      if card.value == '10': 
          value = card.value
          space = ''
      else:
          value = card.value[0]
          space = ' ' 
      suit = suits_name.index(card.suit)
      suit = suits_symbols[suit]

      lines[0].append('┌─────────┐')
      lines[1].append('│{}{}       │'.format(value, space))
      lines[2].append('│         │')
      lines[3].append('│         │')
      lines[4].append('│    {}    │'.format(suit))
      lines[5].append('│         │')
      lines[6].append('│         │')
      lines[7].append('│       {}{}│'.format(space, value))
      lines[8].append('└─────────┘')

    result = []
    for index, line in enumerate(lines):
      if(add_inv == False):
        result.append(''.join(lines[index]))
      else:
        lines[index].append(" ~ * * * ~ ")
        result.append(''.join(lines[index]))

    return '\n'.join(result)


deck = pydealer.Deck()
deck.shuffle()
hand = deck.deal(2)
dealer_v = deck.deal(1)
dealer_nv = deck.deal(1)

pull = deck.deal(1)
hand.add(pull)

hand.cards[0]
print("####### Dealer #######")
print(ascii_version_of_card(dealer_v.cards, True))
print(">>----- Agents -----<<")
print(ascii_version_of_card(hand.cards))

print("######################")






####### Dealer #######
┌─────────┐ ~ * * * ~ 
│10       │ ~ * * * ~ 
│         │ ~ * * * ~ 
│         │ ~ * * * ~ 
│    ♣    │ ~ * * * ~ 
│         │ ~ * * * ~ 
│         │ ~ * * * ~ 
│       10│ ~ * * * ~ 
└─────────┘ ~ * * * ~ 
>>----- Agents -----<<
┌─────────┐┌─────────┐┌─────────┐
│2        ││K        ││A        │
│         ││         ││         │
│         ││         ││         │
│    ♠    ││    ♥    ││    ♣    │
│         ││         ││         │
│         ││         ││         │
│        2││        K││        A│
└─────────┘└─────────┘└─────────┘
######################


In [150]:
import numpy as np

class Table(object): 
  card_values = {
        'Ace_used': 1,
        'Ace': 11,
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
        '10': 10,
        'Jack': 10,
        'Queen': 10,
        'King': 10
    }

  def hit(self):
    new_card = self.deck.deal(1)
    self.hand_sum += self.card_values[new_card[0].value]
    self.hand.add(new_card)

    if(self.hand_sum > 21):
      for i in self.hand:
        if i.value == 'Ace':
          i.value = 'Ace_used'
          self.hand_sum -= 10
          return 

  def check(self):
    self.dealer_v.add(self.dealer_nv)
    self.dealer_sum += (self.card_values[self.dealer_nv[0].value])

    while(self.dealer_sum < 17):
      new_card = self.deck.deal(1)
      self.dealer_sum += self.card_values[new_card[0].value]
      self.dealer_v.add(new_card)

      if(self.dealer_sum > 21):
        for i in self.dealer_v:
          if i.value == 'Ace':
            i.value = 'Ace_used'
            self.dealer_sum -= 10
            break
             
    

  def __str__(self):
    ret = "####### Dealer #######\n"
    ret += ascii_version_of_card(self.dealer_v.cards, False) 
    ret += "\n>>----- Agents -----<<\n"
    ret += ascii_version_of_card(self.hand.cards)
    ret += "\n######################\n"
    return ret

  def __init__(self):
    self.deck = pydealer.Deck()
    self.deck.shuffle()
    self.hand = self.deck.deal(2)
    self.dealer_v = self.deck.deal(1)
    self.dealer_nv = self.deck.deal(1)

    
    self.hand_sum = 0
    for i in self.hand:
      self.hand_sum += self.card_values[i.value]
    self.dealer_sum = 0
    for i in self.dealer_v:
      self.dealer_sum += self.card_values[i.value]

hit_win = np.full((10, 19), 1)
hit_play = np.full((10, 19), 1)
check_win = np.full((10, 19), 1)
check_play = np.full((10, 19), 1)

for _ in range(10000):
  play = True
  k = Table()
  x = k.dealer_sum - 2
  y = k.hand_sum - 4
  while(play):
    #if((hit_win[x, y] / hit_play[x, y] * np.random.rand(1) - check_win[x, y] / check_play[x, y] * np.random.rand(1)) >= 0):
    if(check_win[x, y] / check_play[x, y] > (0.5 + np.random.normal(0, 0.05))):
      k.check()#zameniti sa hit ako se koristi prvi if
    else:
      k.hit()
    if(k.hand_sum > 21 or k.dealer_sum >= 17):
      play = False
      break
    y = k.hand_sum - 4
    
  if(k.hand_sum == k.dealer_sum):
    pass
  elif(k.hand_sum > 21):
    hit_play[x, y] += 1
  elif(k.dealer_sum > 21 or k.dealer_sum < k.hand_sum):
    check_win[x, y] += 1
    check_play[x, y] += 1
  else: #diler veca suma
    check_play[x, y] += 1

  
  """
  print(hit_play)
  print(k)
  print("dil sum")
  print(k.dealer_sum)
  print("agent suma")
  print(k.hand_sum)
  #print(k.hand_sum)
  """




[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


In [ ]:
print(hit_win)
print("\n")
print(hit_play)
print("\n")
print(check_win)
print("\n")
print(check_play)
print("\n")
print((hit_win/hit_play - check_win/check_play).round(2))
print("\n")
print((check_win/check_play).round(2))